# library installation
# change runtime type to GPU for face recognition

In [2]:
!pip install tweepy

# importing libraries

In [3]:
import tweepy
import math
import datetime
import pandas as pd
# import face_recognition
# import urllib
import time
import numpy as np
import cv2
from PIL import Image
import urllib
from urllib.request import urlopen
from google.colab.patches import cv2_imshow
from skimage import io

# Tweepy Auth Object

In [4]:
def establish_connection():
    global auth
    global api
    auth = tweepy.OAuthHandler("Ueyphk6rJS5wFjnFKYxPLuhIr", "9bVtiSWrXOBFPjDwCXZynJ0W8uYpd8KMhsJcRyG63ocD3hzkMM")
    auth.set_access_token('1319706669245292544-wkRstlNWriivHWdxTX7EXkV60JvH9R', 'KibZ5HWNx82bNQ5nyaXNADarY08mzr4t1Vdz3evdHo9Og')
    api = tweepy.API(auth)

establish_connection()

# Function for checking if user is Active

In [5]:
def user_active(screen_name):
  flag = True
  user = 0
  try:
    user = api.get_user(screen_name) 
    followers_count = user.followers_count 
  except tweepy.TweepError as e:
        if str(e).find("Rate limit exceeded") != -1:
            print("Rate limit Error: Execution stopped for 15*61 seconds.")
            print(f"Program paused at    : {datetime.datetime.now()+ datetime.timedelta(minutes=5.5*60)} Indian Standard Time (IST)")
            print(f"Expected resume time : {datetime.datetime.now() + datetime.timedelta(seconds=15*61) + datetime.timedelta(minutes=5.5*60)} Indian Standard Time (IST)")
            print("Program at user_active(screen_name)")
            print(f"params => {screen_name}")
            time.sleep(15 * 61)
            try:
              user = api.get_user(screen_name) 
              followers_count = user.followers_count 
              
            except:
              if str(e).find("User has been suspended") != -1 or str(e).find("User not found") != -1:
                print(f'User "{screen_name}" returned user_suspened/user_not_found error. Action : skipped')  
                flag = False
                
              else:
                flag = False
                print(f'User "{screen_name}" returned error :{str(e)}. Action : skipped')  
                

        elif str(e).find("User has been suspended") != -1 or str(e).find("User not found") != -1:
              flag = False
              print(f'User "{screen_name}" returned user_suspened/user_not_found error. Action : skipped')  
                
        else:
            flag = False
            print(f'User "{screen_name}" returned error :{str(e)}. Action : skipped')  
                
  return [flag,user]

# Fetching User Timeline

In [6]:
############################################
### Call function only if User is active ###
############################################


def user_timeline_fetch(screen_name, count_limit):
    
    # params      ===> screen_name  : accounts screen name without @
    # count_limit ===> max number of tweets to be fecthed (not exact)
    tweets_list = []
    count = 0;
    for i in range(1,100):
        if count > count_limit:
            break
        try:
            time.sleep(0.1)
            time_line = api.user_timeline(screen_name, page = i)
        
        except tweepy.TweepError as e:
            if str(e).find("Rate limit exceeded") != -1:
                print("Rate limit Error: Execution stopped for 15*61 seconds.")
                print(f"Program paused at    : {datetime.datetime.now()+ datetime.timedelta(minutes=5.5*60)} Indian Standard Time (IST)")
                print(f"Expected resume time : {datetime.datetime.now() + datetime.timedelta(seconds=15*61) + datetime.timedelta(minutes=5.5*60)} Indian Standard Time (IST)")
                print("Program at user_timeline_fetch(screen_name)")
                print(f"params => {screen_name}")
                time.sleep(15 * 61)
                try:
                    time_line = api.user_timeline(screen_name, page = i) 
                
                except:
                    if str(e).find("Failed to send request") != -1 :
                        return [104]
                    print(f'User "{screen_name}"\'s timeline returned error :{str(e)}. Action : skipped')
                    return [0]  
                    
            
            else:
                if str(e).find("Failed to send request") != -1 :
                    return [104]
                print(f'User "{screen_name}"\'s timeline returned error :{str(e)}. Action : skipped')  
                return [0]    





        if len(time_line) == 0:
            break
        for tweet in time_line:
            tweets_list.append(tweet)
            count = count + 1
        
    return tweets_list

# Original Tweets V/s Retweets

In [7]:
###########################################################################################
######## Function returns list as follow:: ################################################
## [retweets, original_tweets, tweet-retweet ratio,, overall_freq_percentage_for_week,
##  overall_freq_percentage_for_week, overall_freq_percentage_for_week, overall_freq_for_week, retweet_freq_for_week, original_tweet_freq_for_week]
#######################################################################################


def retweet_perc(tweets_list):
    retweet = 0
    ori_tweet = 0
    retweet_list = []
    ori_list = [] 
    overall_perc = [0, 0, 0, 0, 0, 0, 0]
    retweet_perc = [0, 0, 0, 0, 0, 0, 0]
    ori_perc = [0, 0, 0, 0, 0, 0, 0]
    
    for t in tweets_list:
        p = t.text.find('RT @')
        if p != -1:
            retweet = retweet + 1
            retweet_list.append(t)
        else:
            ori_list.append(t)
            ori_tweet = ori_tweet + 1
    
    retweet_distribution = week_distribution(retweet_list)
    ori_distribution = week_distribution(ori_list)
    overall_distribution = [x + y for x, y in zip(retweet_distribution, ori_distribution)]

    if len(tweets_list) != 0:
        overall_perc = [x / len(tweets_list) for x in overall_distribution]
    if len(retweet_list) != 0:
        retweet_perc = [x / len(retweet_list) for x in retweet_distribution]
    if len(ori_list) != 0:
        ori_perc = [x / len(ori_list) for x in ori_distribution]
    re_ori_ratio = retweet
    if ori_tweet!=0:
        re_ori_ratio /= ori_tweet
    sorted_retweet = retweet_perc
    sorted_retweet.sort()
    return [retweet, ori_tweet,re_ori_ratio] + overall_perc + retweet_perc + ori_perc + overall_distribution + retweet_distribution + ori_distribution + sorted_retweet

# Week Distribution Analysis

In [8]:
def week_distribution(tweets_list):
    # tweets_list : list returned by user_timeline_fetch() 
    week_freq = [0, 0, 0, 0, 0, 0, 0]
    for t in tweets_list:
        x = t.created_at
        week_freq[x.weekday()] = week_freq[x.weekday()] + 1
    return week_freq

# Face detection

In [9]:
def face_flag(user):
    url = user.profile_image_url
    url = url.replace('_normal', '')
    response = urllib.request.urlopen(url)
    image = face_recognition.api.load_image_file(response)
    res = face_recognition.api.face_encodings(image)
    if len(res) == 0:
        return 0
    else: 
        return 1

In [10]:
face_cascade = cv2.CascadeClassifier('/content/drive/MyDrive/haarcascade_frontalface_default.xml')
face_cascade_side = cv2.CascadeClassifier('/content/drive/MyDrive/haarcascade_profile.xml')
body_cascade = cv2.CascadeClassifier('/content/drive/MyDrive/haarcascade_fullbody.xml')


def face_dec(user):
    url = user.profile_image_url
    url = url.replace('_normal', '')
    image = io.imread(url) 
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    face1 = face_cascade.detectMultiScale(gray, 1.3, 5)
    face2 = face_cascade_side.detectMultiScale(gray, 1.3, 5)
    face3 = face_cascade_side.detectMultiScale(cv2.flip(gray, 1), 1.3, 5)
    body  = body_cascade.detectMultiScale(gray, 1.3, 5)

    if len(face1)+len(face2)+len(face3) + len(body) == 0:
        return 0
    else: 
        return 1

# Custom Timestamp clustering Single dimension

In [11]:
#######################################################################################################################
######### Function returns list as [no_of_clusters,no_of_clusters_per_day,no_of_tweets,no_of_tweets_per_day] ##########
#######################################################################################################################


def timestamp_custom_clustering(tweet_list):
    if len(tweet_list)<2:
        return [0,0,0,0,0]
    sec_time = []
    for x in tweet_list:
        sec_time.append((x.created_at.timestamp()/60))

    minx = min(sec_time)
    sec_time = [x - minx for x in sec_time]
    sec_time.sort()
    cluster_count = 0
    cluster_details = []
    flag = True
    for i in range(len(sec_time)):
        # print(i)
        if flag:
            start = i
            end = i
            flag = False
        elif sec_time[i]-sec_time[i-1] <= 1:
            end += 1
        elif flag == False:
            cluster_details.append((start,end))
            start = i
            end = i

        else:
            pass
            #situation never occurs
    cluster_details.append((start,end))
    no_of_clusters = len(cluster_details)
    no_of_tweets = len(sec_time)
    cluster_ratio = no_of_clusters/no_of_tweets
    nod = sec_time[no_of_tweets-1]/(60*24)
    no_of_clusters_per_day = no_of_clusters/nod
    no_of_tweets_per_day = no_of_tweets/nod
    no_of_tweets_per_cluster = no_of_tweets/no_of_clusters

    return [no_of_clusters,no_of_clusters_per_day,no_of_tweets_per_cluster,no_of_tweets,no_of_tweets_per_day]


# Keywords analysis

In [12]:
######### Returns total no of apperances keyword 'follow' ,total no of tweets, and apperances per tweet
def keyword_analysis(tweet_list):
    count = 0;
    total = 0;
    for tweet in tweet_list:
        count += tweet.text.count('follow')
        count += tweet.text.count('Follow')
        count += tweet.text.count('like')
        count += tweet.text.count('Like')
        count += tweet.text.count('tweet')
        count += tweet.text.count('Tweet')
        total += 1
    if total == 0:
        return[0,0]
    return [count, count/total]

# Functions clubbing

In [13]:
def fetch_user_details(screen_name):
    user = user_active(screen_name)
    if not user[0]:
        return []
    # if user[1].statuses_count >= 3:
    #     print("Pruned\n")
    #     return []
    tweet_list = user_timeline_fetch(screen_name, 1000)
    if len(tweet_list) == 1 and tweet_list[0] == 104:
        print("\nRe_fetch tweet list\n")
        time.sleep(1)
        tweet_list = user_timeline_fetch(screen_name, 1000) 
    if len(tweet_list) == 1 and (tweet_list[0] == 0 or tweet_list[0] == 104):
        print("\nTweet list error\n")
        return [] 
    

    retweet_analysis = retweet_perc(tweet_list)
    cluster_analysis = timestamp_custom_clustering(tweet_list)
    keyword_result = keyword_analysis(tweet_list)
    no_followers = user[1].followers_count
    no_following = user[1].friends_count
    description_present = 0
    if user[1].description != '':
        description_present = 1
    profile_image = 0
    if user[1].default_profile_image == True:
        profile_image = 1
    follow_following_ratio = no_followers
    if no_following != 0:
        follow_following_ratio = no_followers/no_following
    face = 0
    if profile_image == 0:
        face = face_dec(user[1])



    return [no_followers,no_following,follow_following_ratio,description_present,profile_image] + retweet_analysis + cluster_analysis + [face] + keyword_result + [0]


# DataFrame Creation

In [14]:
user_details_dataframe = pd.DataFrame(columns=['no_followers', 'no_following','follow_following_ratio', 'description_present', 'profile_image_default','no_of_retweets', 'no_of_original_tweets', 'retweet_original_ratio', 'mon_tweet_probablitity', 'tue_tweet_probablitity', 'wed_tweet_probablitity', 'thru_tweet_probablitity', 'fri_tweet_probablitity', 'sat_tweet_probablitity', 'sun_tweet_probablitity', 'mon_retweet_probablitity', 'tue_retweet_probablitity', 'wed_retweet_probablitity', 'thru_retweet_probablitity', 'fri_retweet_probablitity', 'sat_retweet_probablitity', 'sun_retweet_probablitity', 'mon_original_tweet_probablitity', 'tue_original_tweet_probablitity', 'wed_original_tweet_probablitity', 'thru_original_tweet_probablitity', 'fri_original_tweet_probablitity', 'sat_original_tweet_probablitity', 'sun_original_tweet_probablitity' ,'mon_tweet_freq', 'tue_tweet_freq', 'wed_tweet_freq', 'thru_tweet_freq', 'fri_tweet_freq', 'sat_tweet_freq', 'sun_tweet_freq', 'mon_retweet_freq', 'tue_retweet_freq', 'wed_retweet_freq', 'thru_retweet_freq', 'fri_retweet_freq', 'sat_retweet_freq', 'sun_retweet_freq', 'mon_original_tweet_freq', 'tue_original_tweet_freq', 'wed_original_tweet_freq', 'thru_original_tweet_freq', 'fri_original_tweet_freq', 'sat_original_tweet_freq', 'sun_original_tweet_freq','1_lowest','2_lowest','3_lowest','4_lowest','5_lowest','6_lowest','7_lowest', 'no_of_clusters', 'no_of_clusters_per_day','no_of_tweets_per_cluster', 'no_of_tweets', 'no_of_tweets_per_day', 'face_visible_in_profile', 'keyword_freq', 'keyword_freq_per_tweet','user_type'])


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# df = pd.read_csv('/content/drive/MyDrive/final_users1(1) - final_users1(1).csv')
# user_list = df['users'].values.tolist()
# user_list

In [ ]:
df1 = df[df['label']==1]

In [ ]:
df2 = df1[df1['followers_count']>-1]


In [ ]:
user_list = df2['users'].values.tolist()
# user_list

In [ ]:
len(user_list)

511

In [15]:
i = 0
# user_list = user_list + ['anubhavstardom','MANOJSI29829778','Manishk78164243','DebalalP','GautamKanjilal4','ChandeliyaJatin','Hardikr57256211','Klakshmilakshm1','Enrico50687182','CharelNiranjan','AHNADNAN1','29_nandini','AnkujDey4','GAURAVS18169944','CargoPackersAn1','AdwaitMastud','BildarDev']
user_list = ['Chandra39534859']
for user in user_list:
    try:
        print(f'Index : {i}\n User : {user_list[i]}\n')
        user_obj = fetch_user_details(user)
        if len(user_obj) == 0:
            i += 1
            continue
        user_details_dataframe.loc[i] = user_obj
        user_details_dataframe.to_csv('/content/drive/MyDrive/chandu_foll.csv') 
        i += 1
    except Exception as e:
        print('\n')
        print(f'Error : {str(e)}\n')
        print('Resetting connection')
        establish_connection() 

Index : 0
 User : Chandra39534859



In [16]:
user_details_dataframe

,no_followers,no_following,follow_following_ratio,description_present,profile_image_default,no_of_retweets,no_of_original_tweets,retweet_original_ratio,mon_tweet_probablitity,tue_tweet_probablitity,wed_tweet_probablitity,thru_tweet_probablitity,fri_tweet_probablitity,sat_tweet_probablitity,sun_tweet_probablitity,mon_retweet_probablitity,tue_retweet_probablitity,wed_retweet_probablitity,thru_retweet_probablitity,fri_retweet_probablitity,sat_retweet_probablitity,sun_retweet_probablitity,mon_original_tweet_probablitity,tue_original_tweet_probablitity,wed_original_tweet_probablitity,thru_original_tweet_probablitity,fri_original_tweet_probablitity,sat_original_tweet_probablitity,sun_original_tweet_probablitity,mon_tweet_freq,tue_tweet_freq,wed_tweet_freq,thru_tweet_freq,fri_tweet_freq,sat_tweet_freq,sun_tweet_freq,mon_retweet_freq,tue_retweet_freq,wed_retweet_freq,thru_retweet_freq,fri_retweet_freq,sat_retweet_freq,sun_retweet_freq,mon_original_tweet_freq,tue_original_tweet_freq,wed_original_tweet_freq,thru_original_tweet_freq,fri_original_tweet_freq,sat_original_tweet_freq,sun_original_tweet_freq,1_lowest,2_lowest,3_lowest,4_lowest,5_lowest,6_lowest,7_lowest,no_of_clusters,no_of_clusters_per_day,no_of_tweets_per_cluster,no_of_tweets,no_of_tweets_per_day,face_visible_in_profile,keyword_freq,keyword_freq_per_tweet,user_type
0,5.0,32.0,0.15625,1.0,1.0,2.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.333333,0.666667,0.0,0.0,0.0,0.0,0.0,0.5,0.5,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.5,0.5,3.0,2.820488,1.0,3.0,2.820488,0.0,0.0,0.0,0.0


In [ ]:
for x in user_details_dataframe.columns:
    print(x)

no_followers
no_following
follow_following_ratio
description_present
profile_image_default
no_of_retweets
no_of_original_tweets
retweet_original_ratio
mon_tweet_probablitity
tue_tweet_probablitity
wed_tweet_probablitity
thru_tweet_probablitity
fri_tweet_probablitity
sat_tweet_probablitity
sun_tweet_probablitity
mon_retweet_probablitity
tue_retweet_probablitity
wed_retweet_probablitity
thru_retweet_probablitity
fri_retweet_probablitity
sat_retweet_probablitity
sun_retweet_probablitity
mon_original_tweet_probablitity
tue_original_tweet_probablitity
wed_original_tweet_probablitity
thru_original_tweet_probablitity
fri_original_tweet_probablitity
sat_original_tweet_probablitity
sun_original_tweet_probablitity
mon_tweet_freq
tue_tweet_freq
wed_tweet_freq
thru_tweet_freq
fri_tweet_freq
sat_tweet_freq
sun_tweet_freq
mon_retweet_freq
tue_retweet_freq
wed_retweet_freq
thru_retweet_freq
fri_retweet_freq
sat_retweet_freq
sun_retweet_freq
mon_original_tweet_freq
tue_original_tweet_freq
wed_original

In [ ]:
# url = "https://pbs.twimg.com/profile_images/714005771361656832/MltQXXNH_400x400.jpg"
# url = url.replace('_normal', '')
# image = io.imread(url) 
# gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
# face1 = face_cascade.detectMultiScale(gray, 1.3, 5)
# face2 = face_cascade_side.detectMultiScale(gray, 1.3, 5)
# face3 = face_cascade_side.detectMultiScale(cv2.flip(gray, 1), 1.3, 5)
# body  = body_cascade.detectMultiScale(gray, 1.3, 5)

# if len(face1)+len(face2)+len(face3) + len(body) == 0:
#     print(0)
# else: 
#     print(1)

1
